# ChebNet

## 基本方法     

$$g_\theta(L) = \sum^K_{k=0} \theta_kL^K$$              

1. 由于只计算了$K$次方，因此此时是可局部化的——$K-localized$

2. 仅计算$K$个矩阵的加权和的权重$\theta_k$，因此计算复杂度为$o(K)$

$$g_\theta(\Lambda) = \sum^K_{k=0} \theta_k\Lambda^K$$

$$y =U g_\theta(\Lambda)\hat x = U \sum^K_{k=0} \theta_k\Lambda^KU^T x $$

<font color='red'> 问题：ChebNet有一个问题——即在做谱分解以及计算$\Lambda^K$时会有$o(N^2)$的计算复杂度，该计算复杂度到底出现在哪里 </font>

## 切比雪夫多项式(Chebyshev polynomial)

### 切比雪夫多项式概念

定义在一个递归上的多项式:   

$T_0(x) = 1,\\
T_1(x) = x,\\
T_k\ (x) = 2xT_{k-1}\ (x)-T_{k-2}\ (x)\\
x\in [-1,1]$

### ChebNet中的切比雪夫多项式     

令$\tilde{\Lambda} = \cfrac{2\Lambda}{\lambda_{max}}-I$

$T_0(\tilde{\Lambda}) = I,T_1(\tilde{\Lambda}) = \tilde{\Lambda},T_k\ (\tilde{\Lambda}) = 2\tilde{\Lambda}T_{k-1}\ (\tilde{\Lambda})-T_{k-2}\ (\tilde{\Lambda}),\tilde{\lambda}\in [-1,1]$


- **现在令$\theta'_k$为我们要学习的参数,目的是降低计算复杂度，让$T_{k} (\tilde{\Lambda})$是一个易于计算的多项式**

$$g_\theta(\Lambda) = \sum^K_{k=0} \theta_k\Lambda^K$$

<font color='blue'> $$\rightarrow g_{\theta'}(\tilde{\Lambda}) = \sum^K_{k=0} \theta'_kT_{k} (\tilde{\Lambda})$$</font>


由于

$$
y= g_{\theta'}(\tilde{L}) = \sum^K_{k=0} \theta'_kT_{k} (\tilde{L})x\\
 =  \theta'_0T_{0} (\tilde{L})x+\theta'_1T_{1} (\tilde{L})x+...\theta'_{K-1}T_{K-1} (\tilde{L})x+\theta'_KT_{K} (\tilde{L})x
$$

令$$T_{k} (\tilde{L})x = \bar{x}_k$$

$$
y= g_{\theta'}(\tilde{L}) = \sum^K_{k=0} \theta'_kT_{k} (\tilde{L})x\\
 =  \theta'_0\bar x_{0}+\theta'_1\bar x_{1} +...\theta'_{K-1}\bar x_{K-1}+\theta'_K\bar x_{K} \\
  = [\bar x_{0},\bar x_{1},...\bar x_{K-1},\bar x_{K}]^T[\theta'_0,\theta'_1,...\theta'_{K-1},\theta'_{K}]
$$

在该式子中，计算$\bar x_k = 2\tilde{L}\bar x_{k-1}-\bar x_{k-2}$，由于进行递归的计算，每次计算的复杂度为$o(E)$,$K$次计算总复杂度为$o(KE)$

# GCN

## 基本公式

$$y= g_{\theta'}(\tilde{L}) = \sum^K_{k=0} \theta'_kT_{k} (\tilde{L})x,K=1$$
$$\rightarrow y = \theta'_0x+\theta'_1\tilde{L}x$$

其中$\tilde{L} = \cfrac{2L}{\lambda_{max}}-I$

$$\rightarrow y = \theta'_0x+\theta'_1\tilde{L}x\\
=\theta'_0x+\theta'_1( \cfrac{2L}{\lambda_{max}}-I)x$$

由于拉普拉斯矩阵为$L=I-D^{-1/2}AD^{-1/2}$，**该拉普拉斯矩阵中的$\lambda_{max}\leq 2$**，因此令$\lambda_{max} \approx 2$，上式等于:

$$\rightarrow y = \theta'_0x+\theta'_1(L-I)x\\
 = \theta'_0x-\theta'_1(D^{-1/2}AD^{-1/2})x$$
 
 此时**设**$\theta = \theta'_0 = -\theta'_1$，上式得到:
 
 $$\rightarrow y =\theta'_0x-\theta'_1(D^{-1/2}AD^{-1/2})x\\
 = \theta (I+D^{-1/2}AD^{-1/2})x$$
 
 论文中提出——renormalization trick:$I_N+D^{-1/2}AD^{-1/2}\rightarrow \tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}$
 
 <font color = "red">用该方法处理后$\tilde A$对角线上每个非$1$元素都变为$1$</font>
 
 
-  **最终处理后GCN前向公式可以表示为:**
 
 $$H^{(l+1)} = \sigma(\tilde{D}^{-1/2}\tilde{A}\tilde{D}^{-1/2}H^{(l)}W^{(l)})$$
 
 $$h_v = f(\frac{1}{|N (v)|}\sum_{u \in N(v)}WX_u+b)$$
 
该公式实际上在做的是——***update***一个点的***feature map***：<font color = "red">把$X_u$通过***Transfrom***之后</font>，将它自己和它的邻居节点加起来，取平均，加上一个***bias***，通过一个***activation***，最后得到下一层的值


## GCN的问题    

<font color = "red">1. 深层GCN的Aggregate容易造成过度smooth问题——GCN的子图的***node feature***在经过深层后会收敛到同一个值</font>

<img style="float: center;" src="resource_pic/GNN/GNN_1_25.PNG" width=600 height=600>

**如何处理**——DropEdge